In [1]:
import psycopg2
from psycopg2  import extras
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

DATABASE = 'postgresql'
USER = 'root'
PASSWORD = 'root'
HOST = 'postgres' # shared-networkに所属しているpostgres containerを指定できる
PORT = '5432'
DB_NAME = 'sandbox'

CONNECT_STR = '{}://{}:{}@{}:{}/{}'.format(DATABASE, USER, PASSWORD, HOST, PORT, DB_NAME)
engine = create_engine(CONNECT_STR)

conn = engine.connect()
res = conn.execute("select datname, datdba, encoding, datcollate, datctype from pg_database;")

for row in res:
    print(row)

('postgres', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('sandbox', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('template1', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('template0', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')


In [2]:
# create poptbl
tablename = 'x001_poptbl'
q1 = "SELECT * FROM information_schema.tables WHERE table_name = '{}';".format(tablename)
res = conn.execute(q1)

q2 = """
CREATE TABLE  {} (
    pref_name  varchar(256) NOT NULL,
    population integer NOT NULL
);
""".format(tablename)

if sum(1 for _ in res) == 0:
    conn.execute(q2)
    res = conn.execute(q1)
    sum(1 for _ in res)

In [3]:
# insert data
data = (
    ('徳島', 100),
    ('香川', 200),
    ('愛媛', 150),
    ('高知', 200),
    ('福岡', 300),
    ('佐賀', 100),
    ('長崎', 200),
    ('東京', 400),
    ('群馬', 50)
)

conn.execute("DELETE FROM {};".format(tablename))

metadata = MetaData()
tbl = Table(tablename, metadata,
              Column('pref_name', String, primary_key=False),
              Column('population', Integer),
             )

for i in data:
    ins = tbl.insert().values(pref_name=i[0], population=i[1])
    conn.execute(ins)

res = conn.execute("SELECT * FROM {};".format(tablename))
for i in res:
    print(i)

('徳島', 100)
('香川', 200)
('愛媛', 150)
('高知', 200)
('福岡', 300)
('佐賀', 100)
('長崎', 200)
('東京', 400)
('群馬', 50)


期待する結果

|地方名|人口|
|:--|:--|
|九州|600|
|四国|650|
|その他|450|

In [4]:
# まずcaseを利用して県名を地方名という体系に変換してみる

q = """
select 
    case pref_name
    when '徳島' then '四国'
    when '香川' then '四国'
    when '愛媛' then '四国'
    when '高知' then '四国'
    when '福岡' then '九州'
    when '佐賀' then '九州'
    when '長崎' then '九州'
    else 'その他' end
    as 地方名, 
    population as 人口
    from x001_poptbl;
"""

res = conn.execute(q)
for i in res:
    print(i)

('四国', 100)
('四国', 200)
('四国', 150)
('四国', 200)
('九州', 300)
('九州', 100)
('九州', 200)
('その他', 400)
('その他', 50)


In [5]:
# 見やすくするためにクエリを分割する

subq = """
    case pref_name
    when '徳島' then '四国'
    when '香川' then '四国'
    when '愛媛' then '四国'
    when '高知' then '四国'
    when '福岡' then '九州'
    when '佐賀' then '九州'
    when '長崎' then '九州'
    else 'その他' end
"""

q = f"""
select {subq} as 地方名, sum(population) as 人口
    from {tablename}
    group by 地方名;
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


select 
    case pref_name
    when '徳島' then '四国'
    when '香川' then '四国'
    when '愛媛' then '四国'
    when '高知' then '四国'
    when '福岡' then '九州'
    when '佐賀' then '九州'
    when '長崎' then '九州'
    else 'その他' end
 as 地方名, sum(population) as 人口
    from x001_poptbl
    group by 地方名;

('九州', 600)
('四国', 650)
('その他', 450)
